In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time

import os


In [2]:
input_file="diy_rejection_13-May'25.csv"
output_file="diy_mco_rejection_12th May'25.csv"

In [3]:
#input_file="diy_all_rejection_01st April.csv"
#output_file="diy_mco_all_rejection_31st Mar'25.csv"

In [4]:
rejection_data=pd.read_csv(input_file,low_memory=False,memory_map=True)
len(rejection_data)

9781

In [5]:
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1382706884,cefe73c9-091c-4430-9b0b-b60f9637afd1,INDIVIDUAL,IFSC in bank proof mismatch with IFSC in Penny...,bankPassbook,2025-05-05 18:40:04.000,diy_mco,display_name_update,NaN
1,1373821483,fd162dcb-2235-42c8-833a-984cca760067,PROPRIETORSHIP,Display name has no connection with name in bu...,shopFrontPhoto,2025-05-13 10:22:03.000,diy_mco,gst_update,NaN
2,1949133612,0763f366-751b-417f-9f0b-316e57bcc7d4,PROPRIETORSHIP,Display name has no connection with name in bu...,shopFrontPhoto,2025-05-12 10:54:26.000,diy_mco,diy_upgrade,NaN
3,1182202113,99a2c7a0-66d0-4ba6-87e4-c1ef14cf0494,INDIVIDUAL,Current Account not allowed for Individual mer...,BankStatement,2025-05-12 16:30:06.000,diy_mco,NaN,NaN
4,1272902429,1ae0eaa9-bc47-430a-a0dc-b415f5079c10,INDIVIDUAL,Savings account - Beneficiary name mismatch wi...,BankStatement,2025-05-12 11:46:18.000,diy_mco,NaN,NaN


In [6]:
rejection_data['updated_at']=pd.to_datetime(rejection_data['updated_at'],errors='coerce')
rejection_data.head(5)

,cust_id,lead_id,entity_type,rejected_field_reason,rejected_field_name,updated_at,solution_type,solution_type_level_2,solution_type_level_3
0,1382706884,cefe73c9-091c-4430-9b0b-b60f9637afd1,INDIVIDUAL,IFSC in bank proof mismatch with IFSC in Penny...,bankPassbook,2025-05-05 18:40:04,diy_mco,display_name_update,NaN
1,1373821483,fd162dcb-2235-42c8-833a-984cca760067,PROPRIETORSHIP,Display name has no connection with name in bu...,shopFrontPhoto,2025-05-13 10:22:03,diy_mco,gst_update,NaN
2,1949133612,0763f366-751b-417f-9f0b-316e57bcc7d4,PROPRIETORSHIP,Display name has no connection with name in bu...,shopFrontPhoto,2025-05-12 10:54:26,diy_mco,diy_upgrade,NaN
3,1182202113,99a2c7a0-66d0-4ba6-87e4-c1ef14cf0494,INDIVIDUAL,Current Account not allowed for Individual mer...,BankStatement,2025-05-12 16:30:06,diy_mco,NaN,NaN
4,1272902429,1ae0eaa9-bc47-430a-a0dc-b415f5079c10,INDIVIDUAL,Savings account - Beneficiary name mismatch wi...,BankStatement,2025-05-12 11:46:18,diy_mco,NaN,NaN


In [7]:
current_date = pd.Timestamp(datetime.now().date())
current_date


Timestamp('2025-05-13 00:00:00')

In [8]:
rejection_data=rejection_data[rejection_data['updated_at'].dt.date != current_date]
len(rejection_data)

9781

In [9]:
rejection_data=rejection_data[rejection_data['solution_type_level_2'].isin(['diy_upgrade','']) | rejection_data['solution_type_level_2'].isna()]
len(rejection_data)

5374

In [10]:
rejection_data.loc[~rejection_data['entity_type'].isin(['INDIVIDUAL','PROPRIETORSHIP']),'entity_type']='NON_PROPRIETORSHIP'
rejection_data['entity_type'].value_counts()

entity_type
INDIVIDUAL            3540
PROPRIETORSHIP        1826
NON_PROPRIETORSHIP       8
Name: count, dtype: int64

In [11]:
rejection_data.insert(rejection_data.columns.get_loc('entity_type')+1,'entuty_type_1',np.nan)
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [12]:
rejection_data.loc[:, 'entuty_type_1'] = rejection_data['entity_type']
rejection_data['entuty_type_1'].value_counts()

C:\Users\avnish1.yadav\AppData\Local\Temp\ipykernel_30008\2853859728.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['PROPRIETORSHIP' 'INDIVIDUAL' 'INDIVIDUAL' ... 'PROPRIETORSHIP'
 'INDIVIDUAL' 'INDIVIDUAL']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  rejection_data.loc[:, 'entuty_type_1'] = rejection_data['entity_type']


entuty_type_1
INDIVIDUAL            3540
PROPRIETORSHIP        1826
NON_PROPRIETORSHIP       8
Name: count, dtype: int64

In [13]:
rejection_data['entuty_type_1']=rejection_data['entuty_type_1'].replace(['PROPRIETORSHIP','NON_PROPRIETORSHIP'],'NON_EDC')
rejection_data['entuty_type_1'].value_counts()

entuty_type_1
INDIVIDUAL    3540
NON_EDC       1834
Name: count, dtype: int64

In [14]:
rejection_data.columns

Index(['cust_id', 'lead_id', 'entity_type', 'entuty_type_1',
       'rejected_field_reason', 'rejected_field_name', 'updated_at',
       'solution_type', 'solution_type_level_2', 'solution_type_level_3'],
      dtype='object')

In [15]:
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_MCO'
rejection_data.loc[(rejection_data['entity_type']=='INDIVIDUAL') & (~rejection_data['solution_type_level_2'].isna()),'solution_type']='DIY_ReKYC'
rejection_data.loc[~(rejection_data['entity_type']=='INDIVIDUAL'),'solution_type']='DIY_Upgrade'

In [16]:
rejection_data.to_csv(output_file, index=False)